In [ ]:
1）同一个问题多种选择性的回答，并提供缺省回答的方案；
       2）能通过正则表达式、模式匹配、关键词提取、句法转换等来回答问题；
       3）能通过正则表达式、最近邻分类法或者支持向量机之一或多种方案来提取用户意图；
       4）通过预建的命名实体类型、角色关系、依赖分析等来进行命名实体识别；
       5）基于Rasa NLU的本地基础聊天机器人系统的构建；

In [ ]:
1）同一个问题多种选择性的回答，并提供缺省回答的方案；

In [3]:
bot_template = "BOT : {0}"
user_template = "USER : {0}"
import random
responses = {'statement': ['tell me more!', 
                           'why do you think that?', 
                           'how long have you felt this way?', 
                           'I find that extremely interesting', 
                           'can you back that up?', 'oh wow!', ':)'], 
             'question': ["I don't know :(", 
                          'you tell me!']
            }

def respond(message):
    # Check for a question mark
    if message.endswith("?"):
        # Return a random question
        return random.choice(responses["question"])
    # Return a random statement
    return random.choice(responses["statement"])

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Send a message to the bot
#send_message("hello")

# Send messages ending in a question mark
send_message("what's today's weather?")
send_message("what's today's weather?")

# Send messages which don't end with a question mark
send_message("I love building chatbots")
send_message("I love building chatbots")

USER : what's today's weather?
BOT : I don't know :(
USER : what's today's weather?
BOT : you tell me!
USER : I love building chatbots
BOT : I find that extremely interesting
USER : I love building chatbots
BOT : oh wow!


In [3]:
# Import the random module


name = "Greg"
weather = "cloudy"

# Define a dictionary containing a list of responses for each message
responses = {
  "what's your name?": [
      "my name is {0}".format(name),
      "they call me {0}".format(name),
      "I go by {0}".format(name)
   ],
  "what's today's weather?": [
      "the weather is {0}".format(weather),
      "it's {0} today".format(weather)
    ],
  "default": ["default message"]
}

# Use random.choice() to choose a matching response
def respond(message):
    # Check if the message is in the responses
    if message in responses:
        # Return a random matching response
        bot_message = random.choice(responses[message])
    else:
        # Return a random "default" response
        bot_message = random.choice(responses["default"])
    return bot_message

send_message("what's today's weather?")

USER : what's today's weather?
BOT : it's cloudy today


In [ ]:
2）能通过正则表达式、模式匹配、关键词提取、句法转换等来回答问题；

In [3]:
import re
rules = {'I want (.*)': ['What would it mean if you got {0}', 
                         'Why do you want {0}', 
                         "What's stopping you from getting {0}"], 
         'do you remember (.*)': ['Did you think I would forget {0}', 
                                  "Why haven't you been able to forget {0}", 
                                  'What about {0}', 
                                  'Yes .. and?'], 
         'do you think (.*)': ['if {0}? Absolutely.', 
                               'No chance'], 
         'if (.*)': ["Do you really think it's likely that {0}", 
                     'Do you wish that {0}', 
                     'What do you think about {0}', 
                     'Really--if {0}']
        }

# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for k in rules.keys():
        # Create a match object
        match =re.search(k,message)
        if match is not None:
            # Choose a random response
            response = random.choice(rules[k])
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response, phrase

# Test match_rule
print(match_rule(rules, "do you remember your last birthday"))

('Did you think I would forget {0}', 'your last birthday')


In [4]:
# Define replace_pronouns()
def replace_pronouns(message):

    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me','you',message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my','your',message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your','my',message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you','me',message)

    return message

print(replace_pronouns("my last birthday"))
print(replace_pronouns("when you went to Florida"))
print(replace_pronouns("I had my own castle"))

your last birthday
when me went to florida
i had your own castle


In [5]:
# Define respond()
def respond(message):
    # Call match_rule
    response, phrase = match_rule(rules, message)
    if '{0}' in response:
        # Replace the pronouns in the phrase
        phrase = replace_pronouns(phrase)
        # Include the phrase in the response
        response = response.format(phrase)
    return response

# Send the messages
send_message("do you remember your last birthday")
send_message("do you think humans should be worried about AI")
send_message("I want a robot friend")
send_message("what if you could be anything you wanted")

USER : do you remember your last birthday
BOT : Why haven't you been able to forget my last birthday
USER : do you think humans should be worried about AI
BOT : if humans should be worried about ai? Absolutely.
USER : I want a robot friend
BOT : What would it mean if you got a robot friend
USER : what if you could be anything you wanted
BOT : Do you wish that me could be anything me wanted


In [7]:
#正则表达式匹配 re.search(a,b)
#re.search.group(0)一整句group(1)(.*)部分

In [ ]:
3）能通过正则表达式、最近邻分类法或者支持向量机之一或多种方案来提取用户意图；

In [4]:
import re
keywords = {
            'seasons': ['season', 'th', 'year'], 
            'teams': ['team', 'thx'], 
            'players': ['he', 'him','his'],
            'games': ['game', 'date'],
           }#创建字典
# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent,keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile("|".join(keys))
    
# Print the patterns
print(patterns)

{'seasons': re.compile('season|th|year'), 'teams': re.compile('team|thx'), 'players': re.compile('he|him|his'), 'games': re.compile('game|date')}


In [6]:
responses = {'seasons': 'Wow ,that is a wonderful season!',
             'teams': 'haha,i know you like this team', 
             'players': 'So you want something about him?', 
             'games': 'you wanna check the details of that game?'
            }

# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if re.search(pattern,message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

# Send messages
send_message("season")
send_message("him")
send_message("date")

USER : season
BOT : Wow ,that is a wonderful season!
USER : him
BOT : So you want something about him?
USER : date
BOT : you wanna check the details of that game?


In [12]:
#分为几步，首先是用re.compile()把一个字典的右侧compile ，左边是意图，右边是代表这个意图可能出现的词或者句子
#第二步，遍历pattern字典，搜索出现的词来对应intent
#在responses字典中寻找intent对应的回复

In [ ]:
4）通过预建的命名实体类型、角色关系、依赖分析等来进行命名实体识别；

In [8]:
import spacy
import numpy as np
nlp = spacy.load('en')

In [17]:
## Using spaCy's entity recogniser

# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

text1='The player you want is {0}'.format(conn.request("GET", "/teams/city/Cleveland", headers=headers))
text2='The team you want is {0}'.format(extract_entities('friends called Lebron james who have worked at Google since 2010')['ORG'])
text3='The season you want is {0}'.format(extract_entities('friends called Lebron james who have worked at Google since 2010')['DATE'])

print(extract_entities('friends called Lebron james who have worked at Google since 2010'))
print(text1)
print(text2)
print(text3)

{'DATE': '2010', 'ORG': 'Google', 'PERSON': 'Lebron james'}
The player you want is Lebron james
The team you want is Google
The season you want is 2010


In [ ]:
5）基于Rasa NLU的本地基础聊天机器人系统的构建；
       6）数据库查询并使用自然语言探索数据库内容（提取参数、创建查询、响应）
       7）基于增量过滤器的单轮多次增量查询技术以及甄别否定实体技术
       8）实现状态机的多轮多次查询技术，并能基于语境问题提供解释和回答
       9）处理拒绝、等待状态转换和待定行动的多轮多次查询技术


In [12]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


# Create a trainer
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

# Try it out
print(interpreter.parse("I'm looking for a player called lebron"))

D:\anaconda\envs\MITproject\lib\site-packages\rasa_nlu\training_data\training_data.py:176: UserWarning: Intent 'None' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))
D:\anaconda\envs\MITproject\lib\site-packages\rasa_nlu\training_data\training_data.py:184: UserWarning: Entity 'area' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))


Fitting 2 folds for each of 6 candidates, totalling 12 fits
{'intent': {'name': 'restaurant_search', 'confidence': 0.7557001821568343}, 'entities': [], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.7557001821568343}, {'name': 'greet', 'confidence': 0.06843176366962572}, {'name': 'affirm', 'confidence': 0.06529717729407698}, {'name': 'goodbye', 'confidence': 0.06053391723472587}, {'name': 'hotel_search', 'confidence': 0.031452282218862}, {'name': 'location', 'confidence': 0.01858467742587517}], 'text': "I'm looking for a player called lebron"}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [19]:
print(interpreter.parse("show me Chinese food in the centre of town"))
print(interpreter.parse("I want an Indian restaurant in the west"))
print(interpreter.parse("are there any good pizza places in the center?"))

{'intent': {'name': 'restaurant_search', 'confidence': 0.6535283876964398}, 'entities': [{'start': 8, 'end': 15, 'value': 'chinese', 'entity': 'cuisine', 'confidence': 0.6057065130162513, 'extractor': 'CRFEntityExtractor', 'processors': ['EntitySynonymMapper']}, {'start': 28, 'end': 34, 'value': 'centre', 'entity': 'location', 'confidence': 0.9343784028432665, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.6535283876964398}, {'name': 'location', 'confidence': 0.10818291425067801}, {'name': 'greet', 'confidence': 0.07613746617320412}, {'name': 'hotel_search', 'confidence': 0.06080333368474942}, {'name': 'goodbye', 'confidence': 0.05104784531648469}, {'name': 'affirm', 'confidence': 0.050300052878444315}], 'text': 'show me Chinese food in the centre of town'}
{'intent': {'name': 'restaurant_search', 'confidence': 0.703297262735067}, 'entities': [{'start': 10, 'end': 16, 'value': 'indian', 'entity': 'cuisine', 'confidence': 0.80100846

In [21]:
print(interpreter.parse("show me Chinese food in the centre of town"))

{'intent': {'name': 'restaurant_search', 'confidence': 0.6535283876964398}, 'entities': [{'start': 8, 'end': 15, 'value': 'chinese', 'entity': 'cuisine', 'confidence': 0.6057065130162513, 'extractor': 'CRFEntityExtractor', 'processors': ['EntitySynonymMapper']}, {'start': 28, 'end': 34, 'value': 'centre', 'entity': 'location', 'confidence': 0.9343784028432665, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.6535283876964398}, {'name': 'location', 'confidence': 0.10818291425067801}, {'name': 'greet', 'confidence': 0.07613746617320412}, {'name': 'hotel_search', 'confidence': 0.06080333368474942}, {'name': 'goodbye', 'confidence': 0.05104784531648469}, {'name': 'affirm', 'confidence': 0.050300052878444315}], 'text': 'show me Chinese food in the centre of town'}


In [ ]:
{'intent': {'name': 'restaurant_search', 'confidence': 0.6535283876964398}, 
 'entities': [{'start': 8, 'end': 15, 'value': 'chinese', 'entity': 'cuisine', 'confidence': 0.6057065130162513, 'extractor': 'CRFEntityExtractor', 'processors': ['EntitySynonymMapper']}, {'start': 28, 'end': 34, 'value': 'centre', 'entity': 'location', 'confidence': 0.9343784028432665, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.6535283876964398}, {'name': 'location', 'confidence': 0.10818291425067801}, {'name': 'greet', 'confidence': 0.07613746617320412}, {'name': 'hotel_search', 'confidence': 0.06080333368474942}, {'name': 'goodbye', 'confidence': 0.05104784531648469}, {'name': 'affirm', 'confidence': 0.050300052878444315}], 'text': 'show me Chinese food in the centre of town'}


In [9]:
6）数据库查询并使用自然语言探索数据库内容（提取参数、创建查询、响应）

SyntaxError: invalid character in identifier (<ipython-input-9-e0baf375cbdf>, line 1)

In [10]:
# Import sqlite3
import sqlite3

# Open connection to DB
conn = sqlite3.connect('hotels.db')

# Create a cursor
c = conn.cursor()

# Define area and price
location, price = "east", "mid"
t = (location, price)

# Execute the query
c.execute('SELECT * FROM hotels WHERE location=? AND price=?', t)

# Print the results
print(c.fetchall())

[('Hotel for Dogs', 'mid', 'east', 3)]


In [13]:
import json
import http.client

conn = http.client.HTTPSConnection("rapidapi.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
    'x-rapidapi-key': "46d84eb380mshb6c36e9eac12d4cp1b44b2jsn84cdfef9a77c"
    }

conn.request("GET", "/seasons/{0}".format(2018), headers=headers)

res = conn.getresponse()
data = res.read()

str=data.decode("utf-8")
dict = json.loads(str)
print(dict['api']['seasons'])

['2015', '2016', '2017', '2018', '2019']


In [18]:
import http.client

conn = http.client.HTTPSConnection("rapidapi.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
    'x-rapidapi-key': "46d84eb380mshb6c36e9eac12d4cp1b44b2jsn84cdfef9a77c"
    }

conn.request("GET", "/players/firstName/{0}".format('lebron'), headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"api":{"status":200,"message":"GET players\/firstName\/lebron","results":1,"filters":["playerId","teamId","league","country","lastName","firstName"],"players":[{"firstName":"LeBron","lastName":"James","teamId":"17","yearsPro":"16","collegeName":"St. Vincent-St. Mary HS (OH)","country":"USA","playerId":"265","dateOfBirth":"1984-12-30","affiliation":"St. Vincent-St. Mary HS (OH)\/USA","startNba":"2003","heightInMeters":"2.06","weightInKilograms":"113.4","leagues":{"standard":{"jersey":"23","active":"1","pos":"F"}}}]}}


In [16]:
import http.client

conn = http.client.HTTPSConnection("rapidapi.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
    'x-rapidapi-key': "46d84eb380mshb6c36e9eac12d4cp1b44b2jsn84cdfef9a77c"
    }

conn.request("GET", "/players/firstName/lebron", headers=headers)
conn.request("GET", "/players/firstName/{0}".format(lebron), headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"api":{"status":200,"message":"GET players\/firstName\/lebron","results":1,"filters":["playerId","teamId","league","country","lastName","firstName"],"players":[{"firstName":"LeBron","lastName":"James","teamId":"17","yearsPro":"16","collegeName":"St. Vincent-St. Mary HS (OH)","country":"USA","playerId":"265","dateOfBirth":"1984-12-30","affiliation":"St. Vincent-St. Mary HS (OH)\/USA","startNba":"2003","heightInMeters":"2.06","weightInKilograms":"113.4","leagues":{"standard":{"jersey":"23","active":"1","pos":"F"}}}]}}


In [11]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " where " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect("hotels.db")
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query,t)#这行很重要
    # Return the results
    return c.fetchall()

# Create the dictionary of column names and values
params = {'location':'south','price':'lo'}

# Find the hotels that match the parameters
print(find_hotels(params))

[('Cozy Cottage', 'lo', 'south', 2)]


In [12]:
# Define respond()
responses = [
    "I'm sorry :( I couldn't find anything like that", 
    '{} is a great hotel!', 
    '{} or {} would work!', 
    '{} is one option, but I know others too :)'
]

def respond(message):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find hotels that match the dictionary
    results = find_hotels(params)
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Select the nth element of the responses array
    return responses[n].format(*names)

print(respond("I want an expensive hotel in the south town"))

Grand Hotel is a great hotel!


In [6]:
7）基于增量过滤器的单轮多次增量查询技术以及甄别否定实体技术

SyntaxError: invalid character in identifier (<ipython-input-6-9b2ea913541c>, line 1)

In [13]:
# Define a respond function, taking the message and existing params as input
def respond(message, params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find the hotels
    results = find_hotels(params)
    names = [r[0] for r in results]
    n = min(len(results), 3)
    # Return the appropriate response
    return responses[n].format(*names), params

# Initialize params dictionary
params = {}

# Pass the messages to the bot
for message in ["I want an expensive hotel", "in the north of town"]:
    print("USER: {}".format(message))
    response, params = respond(message, params)
    print("BOT: {}".format(response))

USER: I want an expensive hotel
BOT: Grand Hotel is one option, but I know others too :)
USER: in the north of town
BOT: Bens BnB is a great hotel!


In [14]:
tests= [
    ("no I don't want to be in the south", {'south': False}), 
    ('no it should be in the south', {'south': True}), 
    ('no in the south not the north', {'north': False, 'south': True}), 
    ('not north', {'north': False})
]

# Define negated_ents()
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    # Find the index of the final character of each entity
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    # Initialise a list to store sentence chunks
    chunks = []
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity is preceeded by a negation, give it the key False
                if "not" in chunk or "n't" in chunk:
                    result[ent] = False
                else:
                    result[ent] = True
    return result  

# Check that the entities are correctly assigned as True or False
for test in tests:
    print(negated_ents(test[0]) == test[1])

True
True
True
True


In [ ]:
8）实现状态机的多轮多次查询技术，并能基于语境问题提供解释和回答


In [15]:
def send_message(policy, state, message):
    print("USER : {}".format(message))
    new_state, response = respond(policy, state, message)
    print("BOT : {}".format(response))
    return new_state

def respond(policy, state, message):
    (new_state, response) = policy[(state, interpret(message))]
    return new_state, response

def interpret(message):
    msg = message.lower()
    if 'order' in msg:
        return 'order'
    if 'kenyan' in msg or 'columbian' in msg:
        return 'specify_coffee'
    return 'none'

In [16]:
# Define the INIT state
INIT = 0

# Define the CHOOSE_COFFEE state
CHOOSE_COFFEE = 1

# Define the ORDERED state
ORDERED = 2

# Define the policy rules
policy = {
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
}

# Create the list of messages
messages = [
    "I'd like to become a professional dancer",
    "well then I'd like to order some coffee",
    "my favourite animal is a zebra",
    "kenyan"
]

# Call send_message() for each message
state = INIT
for message in messages:    
    state = send_message(policy, state, message)

USER : I'd like to become a professional dancer
BOT : I'm sorry - I'm not sure how to help you
USER : well then I'd like to order some coffee
BOT : ok, Colombian or Kenyan?
USER : my favourite animal is a zebra
BOT : I'm sorry - would you like Colombian or Kenyan?
USER : kenyan
BOT : perfect, the beans are on their way!


In [ ]:
 9）处理拒绝、等待状态转换和待定行动的多轮多次查询技术

In [23]:
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

In [24]:
def interpret(message):
    msg = message.lower()
    if 'order' in msg:
        return 'order'
    elif 'yes' in msg:
        return 'affirm'
    elif 'no' in msg:
        return 'deny'
    return 'none'

In [28]:
# Define send_message()
def send_message(pending,message):
    print("USER : {}".format(message))
    action, pending_action = policy(interpret(message))
    if action == "do_pending" and pending is not None:
        print("BOT : {}".format(pending))
    else:
        print("BOT : {}".format(action))
    return pending_action
    
# Define send_messages()
def send_messages(messages):
    pending_action = None
    for msg in messages:
        pending_action = send_message(pending_action, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "ok yes please"
])

USER : I'd like to order some coffee
BOT : Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?
USER : ok yes please
BOT : Alright, I've ordered that for you!


In [29]:
import string

def send_message(state, pending, message):
    print("USER : {}".format(message))
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

def interpret(message):
    msg = message.lower()
    if 'order' in msg:
        return 'order'
    if 'kenyan' in msg or 'columbian' in msg:
        return 'specify_coffee'
    if any([d in msg for d in string.digits]):
        return 'number'    
    return 'none'

In [30]:
# Define the states
INIT=0
AUTHED=1
CHOOSE_COFFEE=2
ORDERED=3

# Define the policy rules
policy_rules = {
    (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
    (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Columbian or Kenyan?", None),    
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "kenyan"
])


USER : I'd like to order some coffee
BOT : you'll have to log in first, what's your phone number?
USER : 555-12345
BOT : perfect, welcome back!
BOT : would you like Columbian or Kenyan?
USER : kenyan
BOT : perfect, the beans are on their way!
BOT : would you like Columbian or Kenyan?
